# Solved by GPT

In [2]:
# installs
!pip install node2vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 4.7 MB/s eta 0:00:0000:0100:01
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.9/28.9 MB 3.0 MB/s eta 0:00:0000:0100:01
  Using cached smart_open-6.3.0-py3-none-any.whl (56 kB)


In [1]:
from dotenv import load_dotenv
import json
import os

load_dotenv()

API_KEY = GCP_PROJECT_ID = os.getenv('BLOCKFROST_API_KEY')


In [2]:
# imports
from datetime import datetime

## Step1

In [3]:
from blockfrost import BlockFrostApi, ApiError

api = BlockFrostApi(
    project_id=API_KEY,  # insert your project id here
)

try:
    status = api.health()
except ApiError as e:
    print(e)

In [4]:
stake_pools = api.pools()

for pool in stake_pools:
    print(pool.pool_id, pool.owners)

AttributeError: 'str' object has no attribute 'pool_id'

In [5]:
import requests

url = "https://cardano-mainnet.blockfrost.io/api/v0/pools"
headers = {
    "project_id": API_KEY
}

pools = []
page = 1
while True:
    response = requests.get(url, headers=headers, params={'page': page})
    if response.status_code == 200:
        stake_pools = response.json()
        if not stake_pools:
            break
        for pool in stake_pools:
            pool_id = pool['pool_id']
            pool_url = f"https://cardano-mainnet.blockfrost.io/api/v0/pools/{pool_id}"
            response = requests.get(pool_url, headers=headers)
            if response.status_code == 200:
                pool_info = response.json()
                owners = pool_info['owners']
                pools.append({
                    "pool_id": pool_id,
                    "owners": owners
                })
            else:
                print("Error:", response.status_code)
        page += 1
    else:
        print("Error:", response.status_code)
        break

print(pools)


[{'pool_id': 'pool1z5uqdk7dzdxaae5633fqfcu2eqzy3a3rgtuvy087fdld7yws0xt', 'owners': ['stake1uy89kzrdlpaz5rzu8x95r4qnlpqhd3f8mf09edjp73vcs3qhktrtm']}, {'pool_id': 'pool1pu5jlj4q9w9jlxeu370a3c9myx47md5j5m2str0naunn2q3lkdy', 'owners': ['stake1u98nnlkvkk23vtvf9273uq7cph5ww6u2yq2389psuqet90sv4xv9v']}, {'pool_id': 'pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peuxn84sj3cu0r', 'owners': ['stake1uxxsa2rlkv728pve3ks7kn7c6v68luagdjgwa0medy6vp9suv96q8']}, {'pool_id': 'pool1ddskftmsscw92d7vnj89pldwx5feegkgcmamgt5t0e4lkd7mdp8', 'owners': ['stake1u9w20uzsevkxe6vp2nlmqjjrcj4gvcf560tvufuvrcjpxvgqaqg0t']}, {'pool_id': 'pool1qqqqqdk4zhsjuxxd8jyvwncf5eucfskz0xjjj64fdmlgj735lr9', 'owners': ['stake1uyu7kwazvf97ccvqrd5u6zruzep2yedn9z8mzqvl420rafq78lk3g', 'stake1uy7ccasgje45h004zws56ktp7nvcnktvez5c76hd84n6a6sd2g66l', 'stake1u8gj9ucwx5gune8sx46maayrqykfvuzqrdv6x68ths9raug7taxze', 'stake1u867tezh9puktlvqvg8tyzkxanf3es338sxa6uy0vhltncqel5g6a']}, {'pool_id': 'pool1dmqzwuql5mylffvn7ln3pr9j7kh4gdsssrmma5wgx56f6rtyf42'

In [6]:
len(pools)

100

In [25]:
fetch = {
    "meta": {
        "date": datetime.now().isoformat()
    },
    "data": pools
}

with open('../data/pool_fetch.json', "w") as f:
    json.dump(fetch, f, indent=2)

### Q: is it possible that one stake can own multiple pools?

### A: Yes, it is possible for one stake to own multiple pools. According to the Cardano documentation¹, a stake pool can have multiple owners who pledge their stake to form one larger pool to ensure it is competitive. However, there is still only one stake pool operator who is responsible for stake pool processes.

Quelle: Unterhaltung mit Bing, 19/05/2023
(1) About stake pools, operators, and owners - Cardano. https://docs.cardano.org/development-guidelines/operating-a-stake-pool/about-stake-pools/.
(2) Cardano | Stake Pool Delegation. https://cardano.org/stake-pool-delegation/.
(3) Can I stake my ADA in more than one staking pool at a time ... - Reddit. https://www.reddit.com/r/cardano/comments/kft7w9/can_i_stake_my_ada_in_more_than_one_staking_pool/.
(4) Running multiple pools is more profitable than just a single one .... https://forum.cardano.org/t/running-multiple-pools-is-more-profitable-than-just-a-single-one-this-does-not-lead-to-true-decentralization/39605.

## Step2

In [43]:
def identify_candidates(stakepools, threshold):
    candidates = []

    endpoint = 'https://cardano-mainnet.blockfrost.io/api/v0'

    for pool in stakepools:
        for owner in pool['owners']:
            url = f'{endpoint}/accounts/{owner}/addresses'
            response = requests.get(url, headers=headers)
            addresses = response.json()

            transactions = []
            for address in addresses:
                address = address['address']
                response = requests.get(f'{endpoint}/addresses/{address}/transactions', headers=headers)
                transactions += response.json()

            print(transactions)
            balance = sum([output['amount'] for output in transactions['outputs']])

            if balance > threshold:
                candidates.append(owner)

    return candidates
candidates = identify_candidates(pools, 1)

[{'tx_hash': '871b14fbe5abb6cacc63f922187c4f10ea9499055a972eb5d3d4e8771af643df', 'tx_index': 4, 'block_height': 4490537, 'block_time': 1596059631}, {'tx_hash': 'c0810285e7cffd0ea65851008392d41dd4cdf223d9263ca7a33e28a7e7b410b8', 'tx_index': 1, 'block_height': 4490540, 'block_time': 1596059691}, {'tx_hash': 'f3a0835d9359ed79f8301ba61ff263188c180ffd6dfddaba60a7e31b8366c38e', 'tx_index': 1, 'block_height': 4490544, 'block_time': 1596059771}, {'tx_hash': '841cca81da918feb9fa7257a34630eac95794be712ed3faae6df64f215ce25f2', 'tx_index': 9, 'block_height': 4490550, 'block_time': 1596059891}, {'tx_hash': '36ae277d2e38807775dd6f50c3affaee3700438f45fab2c4cd3a7462463bf343', 'tx_index': 0, 'block_height': 4513649, 'block_time': 1596522791}, {'tx_hash': 'f7c57295fda5181dd0d3ef44b0f35c6c36befdcbeec5a69f16f3ebca56b2f465', 'tx_index': 1, 'block_height': 4599155, 'block_time': 1598246962}, {'tx_hash': 'ce656a9ece8a6506dfe73fa25ef43686df65d5030e745e29169f4c5d85614f52', 'tx_index': 0, 'block_height': 466119

TypeError: list indices must be integers or slices, not str

In [58]:
endpoint = 'https://cardano-mainnet.blockfrost.io/api/v0'


def identify_candidates(stakepools, threshold):
    candidates = []

    for pool in stakepools:
        for owner in pool['owners']:
            url = f'{endpoint}/accounts/{owner}/addresses'
            response = requests.get(url, headers=headers)
            addresses = response.json()

            totals = []
            for address in addresses:
                address = address['address']
                response = requests.get(f'{endpoint}/addresses/{address}/total', headers=headers)
                total = response.json()
                totals.append(int(total['received_sum'][0]['quantity']) - int(total['sent_sum'][0]['quantity']))


            balance = sum([tot for tot in totals])
            print(balance)

            if balance > threshold:
                candidates.append({
                    "owner": owner,
                    "balance": balance
                    })

    return candidates
candidates = identify_candidates(pools, -1)

532413686610
250000000000
296791428775
0
340001634590
1000014453380
700003469441
500000000000
109612018
2625668711
1233494511978
329048009890
50000829029
329039506953
94861134
279903152512
1017288783441
484539889685
574291732
0
500001824863
100001708190
200487386948
125000000000
186363399575
305507616242
61964570978
2006727670
810074733385
0
0
89271685
61315062424
57641897780
10000365423
466144584300
550599918423
557274822146
468878184538
109004969797
136751745222
50001672173
1564213
269183426278
9526704
50167881875
300103494524
520005459360
53557471220
11269190200
101038957791
385057064068
48825699
50000814884
10000492194690
33467635
400357146092
250497200000
109393336492
71728597082
10381265
50544604213
50806862537
50269089944
330039130269
170402207623
50596793859
51133899612
0
387625526894
12377131
256275094405
50268081533
50102777050
167535977447
3216143399481
255447112404
100254732351
580000000000
76001395189
673926007530
147163588242
149552728797
16839798
1202331575457
1063876436

In [59]:
candidates

[{'owner': 'stake1uy89kzrdlpaz5rzu8x95r4qnlpqhd3f8mf09edjp73vcs3qhktrtm',
  'balance': 532413686610},
 {'owner': 'stake1u98nnlkvkk23vtvf9273uq7cph5ww6u2yq2389psuqet90sv4xv9v',
  'balance': 250000000000},
 {'owner': 'stake1uxxsa2rlkv728pve3ks7kn7c6v68luagdjgwa0medy6vp9suv96q8',
  'balance': 296791428775},
 {'owner': 'stake1u9w20uzsevkxe6vp2nlmqjjrcj4gvcf560tvufuvrcjpxvgqaqg0t',
  'balance': 0},
 {'owner': 'stake1uyu7kwazvf97ccvqrd5u6zruzep2yedn9z8mzqvl420rafq78lk3g',
  'balance': 340001634590},
 {'owner': 'stake1uy7ccasgje45h004zws56ktp7nvcnktvez5c76hd84n6a6sd2g66l',
  'balance': 1000014453380},
 {'owner': 'stake1u8gj9ucwx5gune8sx46maayrqykfvuzqrdv6x68ths9raug7taxze',
  'balance': 700003469441},
 {'owner': 'stake1u867tezh9puktlvqvg8tyzkxanf3es338sxa6uy0vhltncqel5g6a',
  'balance': 500000000000},
 {'owner': 'stake1uy9tvq444lvkwtzlalqpn5sswecfat6fgxmham5hushmacs2p9wdq',
  'balance': 109612018},
 {'owner': 'stake1u9h7awdh7aestzrkq0k8krfeesc957kmsmcrr0s3rl8wcfqeg3lgn',
  'balance': 26256687

## Step3

In [77]:
def get_neighbors(address):
    response = requests.get(f'{endpoint}/addresses/{address}/transactions', headers=headers)
    transactions = response.json()

    # get all transactions
    # get all transactions neighbors

    neighbors = set()

    print(transactions)
    for tx in transactions:
        for output in tx.outputs:
            if output.address != address:
                neighbors.add(output.address)

    return list(neighbors)
addr = 'stake1uy89kzrdlpaz5rzu8x95r4qnlpqhd3f8mf09edjp73vcs3qhktrtm'
get_neighbors(addr)

{'status_code': 400, 'error': 'Bad Request', 'message': 'Invalid address for this network or malformed address format.'}


AttributeError: 'str' object has no attribute 'outputs'

## Step4

In [70]:
import networkx as nx

G = nx.Graph()

# add nodes
for candidate in candidates:
    G.add_node(candidate['owner'])

# add edges
for node in G.nodes:
    neighbors = get_neighbors(node)
    for neighbor in neighbors:
        G.add_edge(node, neighbor)

stake1uy89kzrdlpaz5rzu8x95r4qnlpqhd3f8mf09edjp73vcs3qhktrtm


AttributeError: 'str' object has no attribute 'outputs'

In [ ]:
from node2vec import Node2Vec

# Precompute probabilities and generate walks
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=4)

# Embed nodes
model = node2vec.fit(window=10, min_count=1, batch_words=4)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# assuming ground_truth_vector and candidate_vector are your vectors

ground_truth_vector = model.wv.get_vector("ground_truth_node_id") # Replace with actual node id
candidate_vector = model.wv.get_vector("candidate_node_id") # Replace with actual node id

cosine_sim = cosine_similarity([ground_truth_vector], [candidate_vector])

print(f"Cosine similarity between ground truth and candidate: {cosine_sim}")

## Step5

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos = nx.spring_layout(G))
plt.show()

## Step6

## Step7